# 🧬 NGS Data Analysis: Hands-on Practice

**Lecture 4: Next-Generation Sequencing and Genomics**

Instructor: Ho-min Park  
Email: homin.park@ghent.ac.kr | powersimmani@gmail.com

---

## Table of Contents
1. [FASTQ File Format and Quality Scores](#practice-1-fastq-file-format-and-quality-scores)
2. [Quality Control with FastQC](#practice-2-quality-control-with-fastqc)
3. [Read Alignment and SAM/BAM Format](#practice-3-read-alignment-and-sambam-format)
4. [Variant Calling and VCF Format](#practice-4-variant-calling-and-vcf-format)
5. [Variant Annotation and Interpretation](#practice-5-variant-annotation-and-interpretation)
6. [RNA-seq Data Analysis](#practice-6-rna-seq-data-analysis)

---

## Installing and Importing Essential Libraries

In [ ]:
# Install required bioinformatics libraries (run once)
# !pip install biopython pysam pandas matplotlib seaborn numpy

# Import essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO
from Bio.Seq import Seq
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
sns.set_style('whitegrid')

print("✅ All libraries loaded successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

---
## Practice 1: FASTQ File Format and Quality Scores

### 🎯 Learning Objectives
- Understand FASTQ file structure
- Convert Phred quality scores to probability
- Analyze base quality distribution

### 📖 Key Concepts
**FASTQ Format Structure:**
```
@SEQ_ID (Sequence identifier)
GATTTGGGGTTCAAAGCAGTATCGATCAAATAGTAAATCCATTTGTTCAACTCACAGTTT
+ (Separator)
!''*((((***+))%%%++)(%%%%).1***-+*''))**55CCF>>>>>>CCCCCCC65
```

**Phred Quality Score:** Q = -10 × log₁₀(P)  
- Q30 = 99.9% accuracy (1 error in 1,000 bases)
- Q40 = 99.99% accuracy (1 error in 10,000 bases)

In [ ]:
# 1.1 Create simulated FASTQ data
def create_simulated_fastq():
    """Generate simulated FASTQ reads for practice"""
    
    np.random.seed(42)
    
    # Simulate 5 reads
    fastq_data = []
    bases = ['A', 'T', 'G', 'C']
    
    for i in range(5):
        # Generate random sequence
        seq_length = 60
        sequence = ''.join(np.random.choice(bases, seq_length))
        
        # Generate quality scores (ASCII 33-73 = Phred 0-40)
        # Higher quality at the start, lower at the end (typical for Illumina)
        quality_scores = []
        for pos in range(seq_length):
            if pos < 20:
                q = np.random.randint(35, 41)  # High quality
            elif pos < 40:
                q = np.random.randint(30, 36)  # Medium quality
            else:
                q = np.random.randint(20, 31)  # Lower quality
            quality_scores.append(chr(q + 33))
        
        quality_string = ''.join(quality_scores)
        
        # FASTQ entry
        fastq_entry = {
            'id': f'@READ_{i+1}',
            'sequence': sequence,
            'separator': '+',
            'quality': quality_string
        }
        fastq_data.append(fastq_entry)
    
    return fastq_data

# Generate data
fastq_reads = create_simulated_fastq()

# Display first read
print("Example FASTQ Entry:")
print("=" * 70)
read = fastq_reads[0]
print(read['id'])
print(read['sequence'])
print(read['separator'])
print(read['quality'])
print("=" * 70)

In [ ]:
# 1.2 Convert quality scores to Phred scores and error probability
def analyze_quality_scores(quality_string):
    """Convert ASCII quality to Phred scores and error probabilities"""
    
    phred_scores = [ord(char) - 33 for char in quality_string]
    error_probs = [10 ** (-q / 10) for q in phred_scores]
    accuracy = [(1 - p) * 100 for p in error_probs]
    
    return phred_scores, error_probs, accuracy

# Analyze first read
quality_str = fastq_reads[0]['quality']
phred, errors, acc = analyze_quality_scores(quality_str)

# Create results DataFrame
df_quality = pd.DataFrame({
    'Position': range(1, len(quality_str) + 1),
    'ASCII_Char': list(quality_str),
    'Phred_Score': phred,
    'Error_Probability': errors,
    'Accuracy_%': acc
})

print("\nQuality Score Analysis (First 10 positions):")
print(df_quality.head(10).to_string(index=False))

print("\n📊 Summary Statistics:")
print(f"Mean Phred Score: {np.mean(phred):.2f}")
print(f"Minimum Phred Score: {np.min(phred)}")
print(f"Maximum Phred Score: {np.max(phred)}")
print(f"% of bases with Q ≥ 30: {(np.array(phred) >= 30).sum() / len(phred) * 100:.1f}%")

In [ ]:
# 1.3 Visualize quality scores across read positions
def plot_quality_distribution(fastq_data):
    """Plot quality score distribution across all reads"""
    
    # Collect quality scores for each position
    all_qualities = []
    for read in fastq_data:
        phred, _, _ = analyze_quality_scores(read['quality'])
        all_qualities.append(phred)
    
    all_qualities = np.array(all_qualities)
    
    # Calculate statistics per position
    mean_q = np.mean(all_qualities, axis=0)
    q25 = np.percentile(all_qualities, 25, axis=0)
    q75 = np.percentile(all_qualities, 75, axis=0)
    
    # Plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Quality across positions
    positions = range(1, len(mean_q) + 1)
    ax1.plot(positions, mean_q, 'b-', linewidth=2, label='Mean Quality')
    ax1.fill_between(positions, q25, q75, alpha=0.3, color='blue', label='25-75 percentile')
    ax1.axhline(y=30, color='green', linestyle='--', label='Q30 threshold')
    ax1.axhline(y=20, color='orange', linestyle='--', label='Q20 threshold')
    ax1.set_xlabel('Position in read (bp)')
    ax1.set_ylabel('Phred Quality Score')
    ax1.set_title('Per Base Sequence Quality')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Quality score distribution (histogram)
    all_scores_flat = all_qualities.flatten()
    ax2.hist(all_scores_flat, bins=20, color='steelblue', edgecolor='black', alpha=0.7)
    ax2.axvline(x=30, color='green', linestyle='--', linewidth=2, label='Q30')
    ax2.set_xlabel('Phred Quality Score')
    ax2.set_ylabel('Frequency')
    ax2.set_title('Overall Quality Score Distribution')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Quality visualization complete!")
    print(f"Average quality score: {np.mean(all_scores_flat):.2f}")

plot_quality_distribution(fastq_reads)

---
## Practice 2: Quality Control with FastQC

### 🎯 Learning Objectives
- Understand key FastQC metrics
- Identify common quality issues
- Decide on filtering thresholds

### 📖 Key Metrics
- **Per base sequence quality**: Quality drops at read ends
- **Per sequence quality scores**: Overall read quality distribution
- **Sequence duplication levels**: PCR duplicates
- **Adapter content**: Leftover adapter sequences
- **GC content**: Should match expected distribution

In [ ]:
# 2.1 Analyze GC content
def calculate_gc_content(sequence):
    """Calculate GC percentage of a sequence"""
    gc_count = sequence.count('G') + sequence.count('C')
    return (gc_count / len(sequence)) * 100

def analyze_gc_distribution(fastq_data):
    """Analyze GC content distribution across reads"""
    
    gc_contents = []
    for read in fastq_data:
        gc = calculate_gc_content(read['sequence'])
        gc_contents.append(gc)
    
    # Plot distribution
    plt.figure(figsize=(10, 5))
    plt.hist(gc_contents, bins=15, color='teal', edgecolor='black', alpha=0.7)
    plt.axvline(x=50, color='red', linestyle='--', linewidth=2, label='Expected 50%')
    plt.axvline(x=np.mean(gc_contents), color='orange', linestyle='-', linewidth=2, 
                label=f'Mean: {np.mean(gc_contents):.1f}%')
    plt.xlabel('GC Content (%)')
    plt.ylabel('Number of Reads')
    plt.title('GC Content Distribution')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print(f"Mean GC content: {np.mean(gc_contents):.2f}%")
    print(f"Std GC content: {np.std(gc_contents):.2f}%")
    print(f"Range: {np.min(gc_contents):.2f}% - {np.max(gc_contents):.2f}%")
    
    return gc_contents

gc_dist = analyze_gc_distribution(fastq_reads)

In [ ]:
# 2.2 Detect sequence duplicates
def detect_duplicates(fastq_data):
    """Identify duplicate sequences"""
    
    sequence_counts = defaultdict(int)
    for read in fastq_data:
        sequence_counts[read['sequence']] += 1
    
    # Calculate duplication statistics
    total_reads = len(fastq_data)
    unique_reads = len(sequence_counts)
    duplicated_reads = sum(1 for count in sequence_counts.values() if count > 1)
    
    duplication_rate = (total_reads - unique_reads) / total_reads * 100
    
    print("Sequence Duplication Analysis:")
    print("=" * 50)
    print(f"Total reads: {total_reads}")
    print(f"Unique sequences: {unique_reads}")
    print(f"Duplicated sequences: {duplicated_reads}")
    print(f"Duplication rate: {duplication_rate:.2f}%")
    
    # Show most duplicated sequences
    if duplicated_reads > 0:
        print("\nMost duplicated sequences:")
        sorted_seqs = sorted(sequence_counts.items(), key=lambda x: x[1], reverse=True)
        for seq, count in sorted_seqs[:3]:
            if count > 1:
                print(f"  Count: {count}x - {seq[:30]}...")
    
    return sequence_counts

dup_counts = detect_duplicates(fastq_reads)

---
## Practice 3: Read Alignment and SAM/BAM Format

### 🎯 Learning Objectives
- Understand SAM/BAM file structure
- Parse alignment information
- Calculate alignment statistics

### 📖 Key SAM Fields
1. **QNAME** - Read name
2. **FLAG** - Bitwise flag (paired, mapped, etc.)
3. **RNAME** - Reference sequence name (chromosome)
4. **POS** - Alignment position
5. **MAPQ** - Mapping quality score
6. **CIGAR** - Alignment string (M=match, I=insertion, D=deletion)

In [ ]:
# 3.1 Simulate SAM alignment data
def create_simulated_sam_alignments():
    """Generate simulated SAM alignment entries"""
    
    sam_data = [
        {'QNAME': 'READ_1', 'FLAG': 99, 'RNAME': 'chr1', 'POS': 10001, 'MAPQ': 60, 'CIGAR': '76M'},
        {'QNAME': 'READ_2', 'FLAG': 163, 'RNAME': 'chr1', 'POS': 10245, 'MAPQ': 55, 'CIGAR': '70M1I5M'},
        {'QNAME': 'READ_3', 'FLAG': 99, 'RNAME': 'chr2', 'POS': 20500, 'MAPQ': 42, 'CIGAR': '60M2D16M'},
        {'QNAME': 'READ_4', 'FLAG': 4, 'RNAME': '*', 'POS': 0, 'MAPQ': 0, 'CIGAR': '*'},  # Unmapped
        {'QNAME': 'READ_5', 'FLAG': 99, 'RNAME': 'chr1', 'POS': 15780, 'MAPQ': 60, 'CIGAR': '76M'},
    ]
    
    return sam_data

# Create alignments
sam_alignments = create_simulated_sam_alignments()

# Display as DataFrame
df_sam = pd.DataFrame(sam_alignments)
print("SAM Alignment Data:")
print("=" * 80)
print(df_sam.to_string(index=False))
print("=" * 80)

In [ ]:
# 3.2 Parse SAM FLAGS
def decode_sam_flag(flag):
    """Decode SAM bitwise flag"""
    
    flags = {
        1: 'paired',
        2: 'properly paired',
        4: 'unmapped',
        8: 'mate unmapped',
        16: 'reverse strand',
        32: 'mate reverse strand',
        64: 'first in pair',
        128: 'second in pair',
        256: 'secondary alignment',
        512: 'failed QC',
        1024: 'duplicate',
        2048: 'supplementary'
    }
    
    active_flags = []
    for bit, description in flags.items():
        if flag & bit:
            active_flags.append(description)
    
    return active_flags

# Decode FLAGS for all alignments
print("\nFLAG Interpretation:")
print("=" * 80)
for aln in sam_alignments:
    flags = decode_sam_flag(aln['FLAG'])
    print(f"{aln['QNAME']:10s} | FLAG={aln['FLAG']:4d} | {', '.join(flags) if flags else 'No flags'}")

In [ ]:
# 3.3 Parse CIGAR strings
def parse_cigar(cigar_string):
    """Parse CIGAR string and calculate alignment statistics"""
    
    if cigar_string == '*':
        return {'matches': 0, 'insertions': 0, 'deletions': 0, 'total': 0}
    
    import re
    
    operations = re.findall(r'(\d+)([MIDNSHPX=])', cigar_string)
    
    stats = {'matches': 0, 'insertions': 0, 'deletions': 0, 'total': 0}
    
    for length, op in operations:
        length = int(length)
        if op == 'M' or op == '=':
            stats['matches'] += length
        elif op == 'I':
            stats['insertions'] += length
        elif op == 'D':
            stats['deletions'] += length
        
        stats['total'] += length
    
    return stats

# Parse CIGAR for all alignments
print("\nCIGAR String Analysis:")
print("=" * 80)
print(f"{'Read':<12} {'CIGAR':<15} {'Matches':<10} {'Insertions':<12} {'Deletions':<10}")
print("-" * 80)

for aln in sam_alignments:
    stats = parse_cigar(aln['CIGAR'])
    print(f"{aln['QNAME']:<12} {aln['CIGAR']:<15} {stats['matches']:<10} "
          f"{stats['insertions']:<12} {stats['deletions']:<10}")

In [ ]:
# 3.4 Calculate alignment statistics
def calculate_alignment_stats(sam_data):
    """Calculate overall alignment statistics"""
    
    total_reads = len(sam_data)
    mapped_reads = sum(1 for aln in sam_data if not (aln['FLAG'] & 4))
    unmapped_reads = total_reads - mapped_reads
    
    # MAPQ distribution
    mapq_scores = [aln['MAPQ'] for aln in sam_data if aln['MAPQ'] > 0]
    
    # Chromosome distribution
    chr_counts = defaultdict(int)
    for aln in sam_data:
        if aln['RNAME'] != '*':
            chr_counts[aln['RNAME']] += 1
    
    print("Alignment Statistics:")
    print("=" * 50)
    print(f"Total reads: {total_reads}")
    print(f"Mapped reads: {mapped_reads} ({mapped_reads/total_reads*100:.1f}%)")
    print(f"Unmapped reads: {unmapped_reads} ({unmapped_reads/total_reads*100:.1f}%)")
    
    if mapq_scores:
        print(f"\nMapping Quality:")
        print(f"  Mean MAPQ: {np.mean(mapq_scores):.2f}")
        print(f"  High quality (MAPQ ≥ 30): {sum(1 for q in mapq_scores if q >= 30)} reads")
    
    print(f"\nChromosome Distribution:")
    for chr_name, count in sorted(chr_counts.items()):
        print(f"  {chr_name}: {count} reads")
    
    return {
        'total': total_reads,
        'mapped': mapped_reads,
        'unmapped': unmapped_reads,
        'mapping_rate': mapped_reads/total_reads*100
    }

alignment_stats = calculate_alignment_stats(sam_alignments)

---
## Practice 4: Variant Calling and VCF Format

### 🎯 Learning Objectives
- Understand VCF file structure
- Parse variant information
- Calculate variant statistics

### 📖 VCF Format
```
#CHROM  POS     ID      REF  ALT  QUAL  FILTER  INFO           FORMAT  SAMPLE
chr1    10177   .       A    AC   50    PASS    DP=32;AF=0.5   GT:DP   0/1:32
chr1    10352   rs123   T    A    100   PASS    DP=45;AF=1.0   GT:DP   1/1:45
```

**Genotypes:**
- 0/0 = Homozygous reference
- 0/1 = Heterozygous
- 1/1 = Homozygous alternate

In [ ]:
# 4.1 Create simulated VCF data
def create_simulated_vcf():
    """Generate simulated VCF variant data"""
    
    vcf_data = [
        {'CHROM': 'chr1', 'POS': 10177, 'ID': '.', 'REF': 'A', 'ALT': 'G', 
         'QUAL': 50, 'FILTER': 'PASS', 'DP': 32, 'AF': 0.5, 'GT': '0/1'},
        {'CHROM': 'chr1', 'POS': 10352, 'ID': 'rs12345', 'REF': 'T', 'ALT': 'A', 
         'QUAL': 100, 'FILTER': 'PASS', 'DP': 45, 'AF': 1.0, 'GT': '1/1'},
        {'CHROM': 'chr2', 'POS': 20543, 'ID': '.', 'REF': 'GTC', 'ALT': 'G', 
         'QUAL': 35, 'FILTER': 'LowQual', 'DP': 18, 'AF': 0.33, 'GT': '0/1'},
        {'CHROM': 'chr2', 'POS': 20890, 'ID': 'rs67890', 'REF': 'C', 'ALT': 'T', 
         'QUAL': 85, 'FILTER': 'PASS', 'DP': 55, 'AF': 0.5, 'GT': '0/1'},
        {'CHROM': 'chr3', 'POS': 35120, 'ID': '.', 'REF': 'A', 'ALT': 'AGTC', 
         'QUAL': 42, 'FILTER': 'PASS', 'DP': 28, 'AF': 0.5, 'GT': '0/1'},
    ]
    
    return vcf_data

# Create VCF data
vcf_variants = create_simulated_vcf()
df_vcf = pd.DataFrame(vcf_variants)

print("VCF Variant Data:")
print("=" * 100)
print(df_vcf.to_string(index=False))
print("=" * 100)

In [ ]:
# 4.2 Classify variant types
def classify_variant_type(ref, alt):
    """Classify variant as SNV, insertion, or deletion"""
    
    ref_len = len(ref)
    alt_len = len(alt)
    
    if ref_len == alt_len == 1:
        return 'SNV'
    elif ref_len < alt_len:
        return 'Insertion'
    elif ref_len > alt_len:
        return 'Deletion'
    else:
        return 'Complex'

def classify_genotype(gt):
    """Classify genotype as homozygous ref, het, or homozygous alt"""
    
    if gt == '0/0':
        return 'Homozygous Reference'
    elif gt == '0/1' or gt == '1/0':
        return 'Heterozygous'
    elif gt == '1/1':
        return 'Homozygous Alternate'
    else:
        return 'Unknown'

# Add classifications
df_vcf['Variant_Type'] = df_vcf.apply(lambda row: classify_variant_type(row['REF'], row['ALT']), axis=1)
df_vcf['Genotype_Class'] = df_vcf['GT'].apply(classify_genotype)

print("\nVariant Classification:")
print("=" * 100)
print(df_vcf[['CHROM', 'POS', 'REF', 'ALT', 'Variant_Type', 'GT', 'Genotype_Class']].to_string(index=False))

In [ ]:
# 4.3 Analyze variant statistics
def analyze_variant_stats(vcf_data):
    """Calculate comprehensive variant statistics"""
    
    df = pd.DataFrame(vcf_data)
    
    # Add variant types
    df['Variant_Type'] = df.apply(lambda row: classify_variant_type(row['REF'], row['ALT']), axis=1)
    
    total_variants = len(df)
    passed_variants = len(df[df['FILTER'] == 'PASS'])
    
    print("Variant Statistics:")
    print("=" * 50)
    print(f"Total variants: {total_variants}")
    print(f"Passed filters: {passed_variants} ({passed_variants/total_variants*100:.1f}%)")
    print(f"Failed filters: {total_variants - passed_variants}")
    
    # Variant type distribution
    print("\nVariant Type Distribution:")
    type_counts = df['Variant_Type'].value_counts()
    for vtype, count in type_counts.items():
        print(f"  {vtype}: {count} ({count/total_variants*100:.1f}%)")
    
    # Chromosome distribution
    print("\nChromosome Distribution:")
    chr_counts = df['CHROM'].value_counts().sort_index()
    for chrom, count in chr_counts.items():
        print(f"  {chrom}: {count} variants")
    
    # Quality statistics
    print("\nQuality Statistics:")
    print(f"  Mean QUAL: {df['QUAL'].mean():.2f}")
    print(f"  Mean Depth: {df['DP'].mean():.2f}")
    print(f"  Mean Allele Frequency: {df['AF'].mean():.2f}")
    
    return df

df_variants = analyze_variant_stats(vcf_variants)

In [ ]:
# 4.4 Visualize variant distributions
def visualize_variants(df):
    """Create visualizations for variant data"""
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Plot 1: Variant type distribution
    type_counts = df['Variant_Type'].value_counts()
    axes[0, 0].bar(type_counts.index, type_counts.values, color=['steelblue', 'coral', 'lightgreen'])
    axes[0, 0].set_xlabel('Variant Type')
    axes[0, 0].set_ylabel('Count')
    axes[0, 0].set_title('Variant Type Distribution')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: Quality score distribution
    axes[0, 1].hist(df['QUAL'], bins=10, color='purple', edgecolor='black', alpha=0.7)
    axes[0, 1].axvline(x=30, color='red', linestyle='--', linewidth=2, label='Q30 threshold')
    axes[0, 1].set_xlabel('Quality Score')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].set_title('Variant Quality Distribution')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Depth vs Allele Frequency
    passed = df[df['FILTER'] == 'PASS']
    failed = df[df['FILTER'] != 'PASS']
    axes[1, 0].scatter(passed['DP'], passed['AF'], color='green', s=100, alpha=0.6, label='PASS')
    axes[1, 0].scatter(failed['DP'], failed['AF'], color='red', s=100, alpha=0.6, label='Failed')
    axes[1, 0].set_xlabel('Depth (DP)')
    axes[1, 0].set_ylabel('Allele Frequency (AF)')
    axes[1, 0].set_title('Depth vs Allele Frequency')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 4: Chromosome distribution
    chr_counts = df['CHROM'].value_counts().sort_index()
    axes[1, 1].bar(chr_counts.index, chr_counts.values, color='teal', alpha=0.7)
    axes[1, 1].set_xlabel('Chromosome')
    axes[1, 1].set_ylabel('Number of Variants')
    axes[1, 1].set_title('Variants per Chromosome')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Variant visualization complete!")

visualize_variants(df_variants)

---
## Practice 5: Variant Annotation and Interpretation

### 🎯 Learning Objectives
- Understand variant annotation databases
- Predict functional effects
- Filter and prioritize variants

### 📖 Key Annotation Sources
- **gnomAD**: Population frequencies
- **ClinVar**: Clinical significance
- **dbSNP**: Known variant database
- **SIFT/PolyPhen**: Deleteriousness prediction
- **CADD**: Combined prediction scores

In [ ]:
# 5.1 Add simulated annotations
def add_variant_annotations(vcf_data):
    """Add simulated functional annotations to variants"""
    
    annotations = [
        {'Gene': 'BRCA1', 'Effect': 'missense_variant', 'Impact': 'MODERATE', 
         'gnomAD_AF': 0.0001, 'SIFT': 'deleterious', 'ClinVar': 'Pathogenic'},
        {'Gene': 'TP53', 'Effect': 'synonymous_variant', 'Impact': 'LOW', 
         'gnomAD_AF': 0.15, 'SIFT': 'tolerated', 'ClinVar': 'Benign'},
        {'Gene': 'KRAS', 'Effect': 'frameshift_variant', 'Impact': 'HIGH', 
         'gnomAD_AF': None, 'SIFT': 'deleterious', 'ClinVar': 'Likely_pathogenic'},
        {'Gene': 'EGFR', 'Effect': 'missense_variant', 'Impact': 'MODERATE', 
         'gnomAD_AF': 0.002, 'SIFT': 'deleterious', 'ClinVar': 'VUS'},
        {'Gene': 'PIK3CA', 'Effect': 'inframe_insertion', 'Impact': 'MODERATE', 
         'gnomAD_AF': 0.0005, 'SIFT': 'deleterious', 'ClinVar': 'Pathogenic'},
    ]
    
    # Merge with VCF data
    df_vcf = pd.DataFrame(vcf_data)
    df_ann = pd.DataFrame(annotations)
    df_merged = pd.concat([df_vcf, df_ann], axis=1)
    
    return df_merged

# Add annotations
df_annotated = add_variant_annotations(vcf_variants)

print("Annotated Variants:")
print("=" * 120)
display_cols = ['CHROM', 'POS', 'REF', 'ALT', 'Gene', 'Effect', 'Impact', 'gnomAD_AF', 'SIFT', 'ClinVar']
print(df_annotated[display_cols].to_string(index=False))

In [ ]:
# 5.2 Filter and prioritize variants
def prioritize_variants(df):
    """Filter variants based on clinical relevance criteria"""
    
    print("Variant Prioritization Pipeline:")
    print("=" * 60)
    
    total = len(df)
    print(f"Starting variants: {total}")
    
    # Filter 1: Quality
    df_filtered = df[df['FILTER'] == 'PASS'].copy()
    print(f"After quality filter (PASS): {len(df_filtered)} ({len(df_filtered)/total*100:.1f}%)")
    
    # Filter 2: Impact
    df_filtered = df_filtered[df_filtered['Impact'].isin(['HIGH', 'MODERATE'])]
    print(f"After impact filter (HIGH/MODERATE): {len(df_filtered)} ({len(df_filtered)/total*100:.1f}%)")
    
    # Filter 3: Population frequency (rare variants)
    df_filtered = df_filtered[
        (df_filtered['gnomAD_AF'].isna()) | (df_filtered['gnomAD_AF'] < 0.01)
    ]
    print(f"After frequency filter (AF < 1%): {len(df_filtered)} ({len(df_filtered)/total*100:.1f}%)")
    
    # Filter 4: Deleteriousness prediction
    df_filtered = df_filtered[df_filtered['SIFT'] == 'deleterious']
    print(f"After SIFT filter (deleterious): {len(df_filtered)} ({len(df_filtered)/total*100:.1f}%)")
    
    # Prioritize by ClinVar significance
    priority_order = ['Pathogenic', 'Likely_pathogenic', 'VUS']
    df_filtered['Priority'] = df_filtered['ClinVar'].map(
        {val: idx for idx, val in enumerate(priority_order)}
    )
    df_filtered = df_filtered.sort_values('Priority')
    
    print("\n" + "=" * 60)
    print("Prioritized Variants (Highest to Lowest):")
    print("=" * 60)
    
    display_cols = ['Gene', 'Effect', 'Impact', 'gnomAD_AF', 'SIFT', 'ClinVar']
    print(df_filtered[display_cols].to_string(index=False))
    
    return df_filtered

df_prioritized = prioritize_variants(df_annotated)

---
## Practice 6: RNA-seq Data Analysis

### 🎯 Learning Objectives
- Understand RNA-seq count data
- Perform differential expression analysis
- Visualize gene expression patterns

### 📖 Key Concepts
- **TPM/FPKM**: Normalized expression values
- **Fold Change**: Expression ratio between conditions
- **P-value**: Statistical significance
- **FDR**: Multiple testing correction

In [ ]:
# 6.1 Create simulated RNA-seq count data
def create_simulated_rnaseq_data():
    """Generate simulated RNA-seq expression data"""
    
    np.random.seed(42)
    
    genes = ['BRCA1', 'TP53', 'EGFR', 'MYC', 'KRAS', 'PIK3CA', 'PTEN', 'AKT1', 'BRAF', 'NRAS']
    
    # Simulate counts for control vs treatment
    data = []
    for gene in genes:
        # Base expression level
        base_count = np.random.randint(100, 5000)
        
        # Control samples (n=3)
        control_counts = np.random.poisson(base_count, 3)
        
        # Treatment samples (n=3) - some genes up/downregulated
        fold_change = np.random.choice([0.5, 0.8, 1.0, 1.5, 2.0, 3.0])
        treatment_counts = np.random.poisson(base_count * fold_change, 3)
        
        data.append({
            'Gene': gene,
            'Control_1': control_counts[0],
            'Control_2': control_counts[1],
            'Control_3': control_counts[2],
            'Treatment_1': treatment_counts[0],
            'Treatment_2': treatment_counts[1],
            'Treatment_3': treatment_counts[2],
        })
    
    df = pd.DataFrame(data)
    
    # Calculate mean expression
    df['Control_Mean'] = df[['Control_1', 'Control_2', 'Control_3']].mean(axis=1)
    df['Treatment_Mean'] = df[['Treatment_1', 'Treatment_2', 'Treatment_3']].mean(axis=1)
    
    # Calculate fold change
    df['Fold_Change'] = df['Treatment_Mean'] / df['Control_Mean']
    df['Log2_FC'] = np.log2(df['Fold_Change'])
    
    return df

# Generate data
df_rnaseq = create_simulated_rnaseq_data()

print("RNA-seq Expression Data:")
print("=" * 100)
display_cols = ['Gene', 'Control_Mean', 'Treatment_Mean', 'Fold_Change', 'Log2_FC']
print(df_rnaseq[display_cols].round(2).to_string(index=False))

In [ ]:
# 6.2 Perform differential expression analysis
def differential_expression_analysis(df):
    """Simple differential expression analysis using t-test"""
    
    from scipy import stats
    
    results = []
    
    for idx, row in df.iterrows():
        control_samples = [row['Control_1'], row['Control_2'], row['Control_3']]
        treatment_samples = [row['Treatment_1'], row['Treatment_2'], row['Treatment_3']]
        
        # Perform t-test
        t_stat, p_value = stats.ttest_ind(control_samples, treatment_samples)
        
        results.append({
            'Gene': row['Gene'],
            'Log2_FC': row['Log2_FC'],
            'P_value': p_value,
            'Significant': 'Yes' if p_value < 0.05 else 'No',
            'Direction': 'Up' if row['Log2_FC'] > 0 else 'Down'
        })
    
    df_results = pd.DataFrame(results)
    
    print("Differential Expression Results:")
    print("=" * 80)
    print(df_results.round(4).to_string(index=False))
    
    # Summary
    sig_count = (df_results['P_value'] < 0.05).sum()
    up_count = ((df_results['P_value'] < 0.05) & (df_results['Log2_FC'] > 0)).sum()
    down_count = ((df_results['P_value'] < 0.05) & (df_results['Log2_FC'] < 0)).sum()
    
    print("\n" + "=" * 80)
    print(f"Significant genes (p < 0.05): {sig_count}/{len(df_results)}")
    print(f"  Upregulated: {up_count}")
    print(f"  Downregulated: {down_count}")
    
    return df_results

df_de = differential_expression_analysis(df_rnaseq)

In [ ]:
# 6.3 Visualize differential expression
def visualize_differential_expression(df_de):
    """Create volcano plot and expression heatmap"""
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: Volcano plot
    df_de['neg_log10_p'] = -np.log10(df_de['P_value'])
    
    # Color coding
    colors = []
    for _, row in df_de.iterrows():
        if row['P_value'] < 0.05 and abs(row['Log2_FC']) > 1:
            colors.append('red' if row['Log2_FC'] > 0 else 'blue')
        else:
            colors.append('gray')
    
    ax1.scatter(df_de['Log2_FC'], df_de['neg_log10_p'], c=colors, s=100, alpha=0.6)
    ax1.axhline(y=-np.log10(0.05), color='green', linestyle='--', linewidth=2, label='p = 0.05')
    ax1.axvline(x=1, color='orange', linestyle='--', linewidth=2, label='FC = 2')
    ax1.axvline(x=-1, color='orange', linestyle='--', linewidth=2)
    
    # Annotate significant genes
    for _, row in df_de.iterrows():
        if row['P_value'] < 0.05 and abs(row['Log2_FC']) > 1:
            ax1.text(row['Log2_FC'], row['neg_log10_p'], row['Gene'], 
                    fontsize=9, ha='right')
    
    ax1.set_xlabel('Log2 Fold Change', fontsize=12)
    ax1.set_ylabel('-Log10 P-value', fontsize=12)
    ax1.set_title('Volcano Plot', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Expression bar plot
    df_sig = df_de[df_de['P_value'] < 0.05].sort_values('Log2_FC')
    if len(df_sig) > 0:
        colors_bar = ['red' if x > 0 else 'blue' for x in df_sig['Log2_FC']]
        ax2.barh(df_sig['Gene'], df_sig['Log2_FC'], color=colors_bar, alpha=0.7)
        ax2.axvline(x=0, color='black', linewidth=1)
        ax2.set_xlabel('Log2 Fold Change', fontsize=12)
        ax2.set_ylabel('Gene', fontsize=12)
        ax2.set_title('Significant Differentially Expressed Genes', fontsize=14, fontweight='bold')
        ax2.grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Differential expression visualization complete!")

visualize_differential_expression(df_de)

---
## 🎯 Practice Complete!

### Summary of What We Learned:

1. **FASTQ Format**: Understanding quality scores and Phred encoding
2. **Quality Control**: Analyzing read quality, GC content, and duplicates
3. **SAM/BAM Format**: Parsing alignment data and calculating statistics
4. **Variant Calling**: Understanding VCF format and variant classification
5. **Variant Annotation**: Functional prediction and clinical interpretation
6. **RNA-seq Analysis**: Gene expression quantification and differential expression

### Key Insights:
- NGS data analysis follows a structured pipeline from raw reads to biological insights
- Quality control is critical at every step of the analysis
- Proper annotation and interpretation are essential for clinical applications
- Understanding file formats (FASTQ, SAM/BAM, VCF) is fundamental

### Next Steps:
- Practice with real NGS datasets from public databases (NCBI SRA, ENA)
- Learn command-line tools: FastQC, BWA, GATK, SAMtools
- Explore Galaxy platform for web-based analysis
- Try advanced analyses: ChIP-seq, ATAC-seq, single-cell RNA-seq

### Resources:
- **Galaxy**: https://usegalaxy.org
- **GATK Best Practices**: https://gatk.broadinstitute.org
- **Bioconductor**: https://bioconductor.org
- **Training Materials**: https://training.galaxyproject.org

---

**Thank you for completing this hands-on practice!** 🧬🔬

For questions or feedback, contact:  
**Ho-min Park**  
📧 homin.park@ghent.ac.kr | powersimmani@gmail.com